In [95]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import sqlite3
from tqdm import tqdm

In [41]:
data = pd.read_csv("./AB_NYC_2019.csv")

data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [42]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

None

## Предобработка текста

#### Убрем строки с пустым названием

In [43]:
data = data.loc[data['name'].notnull()]

In [44]:
def remove_punctuation(sentence):
    for ch in string.punctuation:
        sentence = sentence.replace(ch,"")
    return sentence.lower()
data['name'] = data.apply(lambda row: remove_punctuation(row['name']), axis=1)
display(data.head(5))

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,clean quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,skylit midtown castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,the village of harlemnew york,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,cozy entire floor of brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,entire apt spacious studioloft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [45]:
data['name'] = data.apply(lambda row: word_tokenize(row['name'], language='english'), axis=1)
display(data.head(5))

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,"[clean, quiet, apt, home, by, the, park]",2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,"[skylit, midtown, castle]",2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,"[the, village, of, harlemnew, york]",4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,"[cozy, entire, floor, of, brownstone]",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,"[entire, apt, spacious, studioloft, by, centra...",7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [46]:
stop_words = stopwords.words('english')
def remove_stop_words(sentence):
    return [w for w in sentence if not w in stop_words]

data['name'] = data.apply(lambda row: remove_stop_words(row['name']), axis=1)
display(data.head(5))

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,"[clean, quiet, apt, home, park]",2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,"[skylit, midtown, castle]",2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,"[village, harlemnew, york]",4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,"[cozy, entire, floor, brownstone]",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,"[entire, apt, spacious, studioloft, central, p...",7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [47]:
lemmatizer = WordNetLemmatizer()
data['name'] = data.apply(lambda row: [lemmatizer.lemmatize(word) for word in row['name']], axis=1)
display(data.head(5))

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,"[clean, quiet, apt, home, park]",2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,"[skylit, midtown, castle]",2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,"[village, harlemnew, york]",4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,"[cozy, entire, floor, brownstone]",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,"[entire, apt, spacious, studioloft, central, p...",7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


## Информационный поиск

In [48]:
con = sqlite3.connect('search_index.db')

In [49]:
con.execute('create table wordlist(word)')
con.execute('create table wordlocation(urlid,wordid,location)')
con.execute('create index wordidx on wordlist(word)')
con.commit()

OperationalError: table wordlist already exists

In [50]:
def entry(table, field, value):
    cur = con.execute("select rowid from %s where %s='%s'" % (table, field, value))
    res = cur.fetchone()
    if res is None:
        cur = con.execute("insert into %s (%s) values ('%s')" % (table, field, value))
        return cur.lastrowid
    else:
        return res[0]

In [51]:
# Добавляем в индекс
for _, apartment in data.iterrows():
    for i, word in enumerate(apartment['name']):
        theWord = entry('wordlist', 'word', word)
        con.execute("insert into wordlocation(urlid,wordid,location) values (%d,%d,%d)" % (apartment['id'], theWord, i))

In [76]:
def get_match_rows(q):
    # заведём строки для дальнейшего формирования запроса
    fieldlist='w0.urlid'
    tablelist=''
    clauselist=''

    wordids=[]

    # разобъём запрос на отдельные слова
    words=[lemmatizer.lemmatize(w) for w in q.lower().split()]
    tablenumber=0

    for word in words:
        # получим ID слова из запроса
        wordrow=con.execute("select rowid from wordlist where word='%s'" % word).fetchone()
        if wordrow!=None:
            wordid=wordrow[0]
            wordids.append(wordid)

            if tablenumber>0: # это для случаев, когда в запросе более одного слова
                tablelist+=','
                clauselist+=' and '
                clauselist+='w%d.urlid=w%d.urlid and ' % (tablenumber-1,tablenumber)

            fieldlist+=',w%d.location' % tablenumber
            tablelist+='wordlocation w%d' % tablenumber
            clauselist+='w%d.wordid=%d' % (tablenumber,wordid)
            tablenumber+=1

    # сформировать запрос в базу
    fullquery='select %s from %s where %s' % (fieldlist,tablelist,clauselist)
    cur=con.execute(fullquery)
    rows=[row for row in cur]

    return rows,wordids

In [77]:
get_match_rows("bright room")

([(94035, 3, 4),
  (100002, 3, 4),
  (106363, 0, 1),
  (458377, 0, 4),
  (565814, 0, 2),
  (622410, 3, 1),
  (637228, 1, 2),
  (794496, 0, 2),
  (823631, 0, 3),
  (848707, 0, 1),
  (924658, 0, 2),
  (940116, 0, 1),
  (1089619, 0, 2),
  (1123737, 1, 4),
  (1150398, 0, 1),
  (1180157, 2, 1),
  (1349237, 0, 3),
  (1391024, 0, 2),
  (1794250, 1, 3),
  (2106706, 1, 2),
  (2145620, 0, 3),
  (2345266, 0, 1),
  (2431607, 0, 2),
  (2451647, 0, 2),
  (2464673, 1, 4),
  (2536005, 0, 1),
  (2791758, 0, 2),
  (2936192, 0, 2),
  (2939670, 0, 1),
  (2942129, 0, 2),
  (3199681, 0, 4),
  (3238355, 1, 3),
  (3278691, 1, 2),
  (3550298, 0, 3),
  (4038794, 0, 1),
  (4116403, 0, 1),
  (4277614, 3, 1),
  (4404544, 1, 3),
  (4824211, 0, 2),
  (4991895, 0, 2),
  (5191965, 0, 1),
  (5588253, 0, 2),
  (5729222, 1, 2),
  (5881122, 1, 3),
  (5926953, 2, 4),
  (6105140, 0, 1),
  (6314794, 0, 1),
  (6402424, 0, 2),
  (6452788, 1, 2),
  (6705558, 1, 2),
  (6763899, 0, 2),
  (6767047, 0, 2),
  (6782841, 0, 2),
  (679

## Исправление опечаток

In [66]:
from spellchecker import SpellChecker

In [72]:
py_spellchecker = SpellChecker()
def fix_spelling(sentence):
    return ' '.join([py_spellchecker.correction(word) for word in sentence.split()])

In [73]:
fix_spelling('anksiety of defil')

'anxiety of devil'

## Ранжирование поисковой выдачи

In [69]:
def normalize_scores(scores,smallIsBetter=0):
    vsmall=0.00001 # Avoid division by zero errors
    if smallIsBetter:
        minscore=min(scores.values(  ))
        return dict([(u,float(minscore)/max(vsmall,l)) for (u,l) in scores.items(  )])
    else:
        maxscore=max(scores.values(  ))
        if maxscore==0: maxscore=vsmall
        return dict([(u,float(c)/maxscore) for (u,c) in scores.items(  )])

In [70]:
def frequency_score(rows):
    counts=dict([(row[0],0) for row in rows])
    for row in rows: counts[row[0]]+=1
    return normalize_scores(counts)

In [71]:
def get_score_list(rows):
    totalscores=dict([(row[0],0) for row in rows])

    weights=[(1.0, frequency_score(rows))]

    for (weight,scores) in weights:
        for url in totalscores:
            totalscores[url]+=weight*scores[url]

    return totalscores

In [74]:
def query(q, dataset, count = 15):
    q = fix_spelling(q)
    rows, wordids = get_match_rows(q)
    scores = get_score_list(rows)
    rankedscores = sorted([(score, url) for (url, score) in scores.items()], reverse=True)[0:count]
    df_res = pd.DataFrame()
    for (score, theId) in rankedscores:
        result = dataset.loc[dataset.id == theId]
        q = {'score': score, 'id': theId, 'neighbourhood_group': result.neighbourhood_group, 'name': result.name}
        df_res =  pd.concat([df_res, pd.DataFrame(q)])
    return df_res

In [78]:
df = query("Brigt room", data)
display(df)

,score,id,neighbourhood_group,name
38088,1.0,30071646,Brooklyn,"[bright, spacious, brooklyn, room, living, room]"
17608,1.0,13848199,Manhattan,"[bright, room, spacious, bright, apartment]"
48536,0.5,36311265,Brooklyn,"[bright, sunny, large, bk, room, shared, space]"
48503,0.5,36298737,Brooklyn,"[fresh, bright, private, room, near, j, train]"
48479,0.5,36278966,Manhattan,"[cozy, bright, room, half, bath, bunk, bed]"
48221,0.5,36142853,Brooklyn,"[bright, cozy, room, greenpoint, loft]"
48098,0.5,36078765,Brooklyn,"[bright, bushwick, room, view, park]"
47970,0.5,36006640,Brooklyn,"[bright, cozy, room, renovated, williamsburg, ..."
47501,0.5,35770687,Brooklyn,"[bright, quiet, room, 25, min, downtown, manha..."
47268,0.5,35668212,Brooklyn,"[bright, shared, room, newly, renovated, house]"


## Рекомендация ранжированного переченя статей из Википедии для каждого документа из коллекции

In [79]:
import warnings
warnings.filterwarnings('ignore')
import wikipedia

In [80]:
def search_in_wikipedia(text):
    wiki_pages = []
    pages = wikipedia.search(text)
    for page in pages:
        try:
            page = wikipedia.page(page)
            wiki_pages.append(page)
        except wikipedia.exceptions.DisambiguationError as e:
            pass
        except wikipedia.exceptions.PageError:
            pass
    return wiki_pages

In [81]:
def get_wiki_article(vehicle_title):
    ans = pd.DataFrame(columns=['wiki_page_name', 'wiki_page_link'])
    for i, page in enumerate(search_in_wikipedia(vehicle_title)):
        ans = pd.concat([ans, pd.DataFrame({'wiki_page_name': page.title, 'wiki_page_link': page.url}, index=[i])])
    return ans

In [82]:
get_wiki_article('New York Brooklyn')

,wiki_page_name,wiki_page_link
0,Blackout cake,https://en.wikipedia.org/wiki/Blackout_cake
1,Brooklyn Dodgers,https://en.wikipedia.org/wiki/Brooklyn_Dodgers
2,Brooklyn Beckham,https://en.wikipedia.org/wiki/Brooklyn_Beckham
3,Brooklyn Nine-Nine,https://en.wikipedia.org/wiki/Brooklyn_Nine-Nine
4,Brooklyn (film),https://en.wikipedia.org/wiki/Brooklyn_(film)


In [89]:
wiki_search_data = data['neighbourhood'].unique()

In [96]:
wiki_pages = dict()
for neighbourhood in tqdm(wiki_search_data):
    wiki_pages[neighbourhood] = get_wiki_article( 'New York' + neighbourhood)

100%|██████████| 221/221 [25:38<00:00,  6.96s/it]


In [97]:
pd.options.display.max_colwidth = 150

In [98]:
display(wiki_pages['Clinton Hill'])

,wiki_page_name,wiki_page_link
0,"Clinton Hill, Brooklyn","https://en.wikipedia.org/wiki/Clinton_Hill,_Brooklyn"
1,"Flower Hill, New York","https://en.wikipedia.org/wiki/Flower_Hill,_New_York"
2,"Manhasset Hills, New York","https://en.wikipedia.org/wiki/Manhasset_Hills,_New_York"
3,"Clinton Hill, Newark","https://en.wikipedia.org/wiki/Clinton_Hill,_Newark"
4,"North Hills, New York","https://en.wikipedia.org/wiki/North_Hills,_New_York"
5,"East Hills, New York","https://en.wikipedia.org/wiki/East_Hills,_New_York"
6,George Clinton (vice president),https://en.wikipedia.org/wiki/George_Clinton_(vice_president)
7,College Hill (New York),https://en.wikipedia.org/wiki/College_Hill_(New_York)
8,"Harbor Hills, New York","https://en.wikipedia.org/wiki/Harbor_Hills,_New_York"
